In [1]:
import torch
import torch.nn as nn
import numpy as np
from datasets import Dataset

In [55]:
rng = np.random.default_rng(seed=2022)
train_size = 50
test_size = 10
#train_size = train_size+test_size

X_0 = {"loc":(1,1), "scale":1}
X_1 = {"loc":(-1,1), "scale":2}
eps = {"loc":(0,0), "scale":0.2}

f_transform = np.array([[ -1,  0],
                        [0.5,0.5]])

def f_bijection(array_of_points):
    return array_of_points @ f_transform - 1

def gen_features(distribution, size=1, transform=False):
    features = rng.normal(**distribution, size=(size,2))
    if transform:
        return f_bijection(features)
    else:
        return features
    
def prep_data(feature_list, label_list):
    task = np.zeros([0,3])
    for idx, features in enumerate(feature_list):
        #print(features, label_list[idx], np.ones([features.shape[0],1]) * label_list[idx])
        datapoints = np.concatenate(
            [features, 
             np.ones([features.shape[0],1]) * label_list[idx]], 
            axis=1)
        #print(datapoints)
        task = np.concatenate([task, datapoints])
        #print(task)
    rng.shuffle(task)
    return task

In [2]:


#Adatgenerálás
#Ezt biztos lehetett volna sokkal elegánsabban
noise1 = rng.normal(**eps, size=(train_size,2))
noise2 = rng.normal(**eps, size=(train_size,2))

Task1_0 = np.concatenate(
    [rng.normal(**X_0, size=(train_size,2)), 
    np.zeros([train_size,1])], 
    axis=1)
Task1_1 = np.concatenate(
    [rng.normal(**X_1, size=(train_size,2)), 
    np.ones([train_size,1])], 
    axis=1)
Task1 = np.concatenate([Task1_0, Task1_1])
rng.shuffle(Task1)
Task1 = torch.from_numpy(Task1)

TaskA_0 = np.concatenate(
    [f_bijection(rng.normal(**X_0, size=(train_size,2))) + noise1, 
    2*np.ones([train_size,1])], 
    axis=1)
TaskA_1 = np.concatenate(
    [f_bijection(rng.normal(**X_1, size=(train_size,2))) + noise2, 
    3*np.ones([train_size,1])], 
    axis=1)
TaskA = np.concatenate([TaskA_0, TaskA_1])
rng.shuffle(TaskA)
TaskA = torch.from_numpy(TaskA)

TaskB_0 = np.concatenate(
    [f_bijection(rng.normal(**X_0, size=(train_size,2))) + 10*noise1, 
    2*np.ones([train_size,1])], 
    axis=1)
TaskB_1 = np.concatenate(
    [f_bijection(rng.normal(**X_1, size=(train_size,2))) + 10*noise2, 
    3*np.ones([train_size,1])], 
    axis=1)
TaskB = np.concatenate([TaskB_0, TaskB_1])
rng.shuffle(TaskB)
TaskB = torch.from_numpy(TaskB)

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
#device = "cpu"

In [4]:
class MockupNetwork(nn.Module):
    def __init__(self, num_classes):
        super(MockupNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(2, 32),
            nn.ReLU(),
            nn.Linear(32, 32),
            nn.ReLU(),
            nn.Linear(32, num_classes),
        )
        self.softmax = nn.Softmax()

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        probabilities = self.softmax(logits)
        return probabilities

In [5]:
model = MockupNetwork(4).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [6]:
def train(data, model, loss_fn, optimizer):
    model.train()
    num_datapoints = data.shape[0]
    for i in range(num_datapoints):
        X, y = data[i,:-1], data[i,-1]
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        #if i % 10 == 0:
        #    loss, current = loss.item(), i * len(X)
        #    print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [7]:
def test(test, model, loss_fn):
    num_datapoints = test.shape[0]
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for i in range(num_datapoints):
            X, y = data[i,:-1], data[i,-1]
            X, y = X.to(device), y.to(device)

            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float)
    test_loss /= num_datapoints
    print(test_loss, correct)

In [9]:
model = model
train(Task1, model, loss_fn, optimizer)

RuntimeError: expected scalar type Double but found Float